In [1]:
# requirements.txt should already be installed

## this is a demo notebook to introduce basic langchain abilities

### in this notebook, we will be playing around with langchain for building nerdoscope - a basic horoscope generator for nerds 🤓

In [1]:
# Make sure you pull a model from ollama using `ollama pull <model-name>` (e.g., phi)
from langchain_community.llms import Ollama

llm = Ollama(model="orca-mini:3b")

In [3]:
# Sanity check: Say hey to your local LLM!

llm.invoke("Hey!")

' Hello! How can I assist you today?'

In [4]:
# Naively ask for a nerdoscope reading

role = "🧑‍💻 Software Develpoer"
query = f'Please generate a funny "nerdoscope" (horoscope for nerds) message for a {role}. Keep the answer short, 5 sentences max.'

for chunks in llm.stream(query):
    print(chunks, end="", flush=True)

 Your nerdoscope for today is as follows: You have a tendency to be a bit too immersed in your work, which can sometimes lead to you missing out on the fun things happening around you. However, with a little focus and determination, you're sure to excel in the world of tech and leave a lasting impact on those around you.

In [5]:
# Let's improve this a bit by using system prompt as context
# This helps the model understand the task better

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a horoscope bot, that provides a "nerdoscope" (horoscope for nerds) reading for a personnas of the tech industry.'
Given the user's role, create a nerdoscope readings that is funny and relatable to their role.
Keep the answer short, 5 sentences max.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{role}")
])

# We'll now use the prompt and llm to create a chain, this is called LCEL (Langchain Expression Language)
chain = prompt | llm

role = "🧑‍💻 Software Develpoer"
for s in chain.stream({"role": role}):
    print(s, end="", flush=True)

 Your nerdoscope reading for the tech industry is: "Your life will be consumed by code until the end of time, but don't worry, at least you get to keep all the intellectual property rights."

In [6]:
# Nice! Let's organize this into a function

from langchain_core.runnables.base import Runnable


def stream_print(chain, context):
    # This function will stream the output of the chain and print it in a readable way for the notebook
    word_count = 0
    buffer = ""

    for s in chain.stream(context):
        buffer += s
        words = buffer.rsplit(' ', 1)
        if len(words) > 1:
            for word in words[:-1]:
                print(word, end=' ', flush=True)
                word_count += 1
                if word_count == 10:
                    print()
                    word_count = 0
            buffer = words[-1]
    if buffer:
        print(buffer, end='', flush=True)
    print()

In [7]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a horoscope bot for personnas in the software development industry.
Given a user's role, generate a personalized "nerdoscope" (horoscope for nerds) message for the week.
Guidelines for the message:
* The message should consists of a single paragraph, 5 sentences max.
* The message should be in the spirit of a horoscope (refer to it as nerdoscope), offering a blend of foresight and advice relevant to the user role, but in a humorous way.
* If possible, the foresight and/or advice should reference a popular tool or technology in the software development industry, relevant to the role.
* The tone should be fun and lighthearted.
* The message should be funny and sarcastic, and personalized to the role.
* The message should include technical jargon.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{role}")
])

chain = prompt | llm

role = "🧑‍💻 Software Develpoer"
stream_print(chain, {"role": role})

 Hey there! I see you're feeling a bit down 
lately due to some technical issues on your project. Don't 
worry, I have a few tips and tricks up my 
sleeve that might help perk you up. For instance, if 
you could use more caffeine in your workflow, why not 
try incorporating some Reddit AMAs into your development process? Just 
be sure to keep a skeptical eye on any particularly 
bold predictions made by your colleagues.


In [24]:
#  Let's start seeing the power of chains!

from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser, RetryOutputParser, OutputFixingParser
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

llm = Ollama(model="orca-mini:3b")
# let's switch to gpt-3.5 hosted in the cloud ☁️🚀
# everything else stays the same!
# load_dotenv()
# llm = AzureChatOpenAI(
#     api_version="2024-02-15-preview",
#     azure_deployment="ss_gpt-35-turbo",
# )

# let's make the output of nerdoscope more interesting

class Nerdoscope(BaseModel):
    reading: str = Field(description="A nerdoscope reading for the role")
    bugs: int = Field(description="A number of bugs that the user will encounter in the upcoming week")
    coffee_cups: int = Field(description="A number of coffee cups the user will drink in the upcoming week")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Nerdoscope)
# fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

prompt = PromptTemplate(
    template="{system_prompt}\n{format_instructions}\n{role}",
    input_variables=["role"],
    partial_variables={
        'system_prompt': system_prompt, 
        'format_instructions': parser.get_format_instructions()
        }
)

chain = prompt | llm | parser

role = "🧑‍💻 Software Develpoer"

nerdoscope: Nerdoscope = chain.invoke({"role": role})

print("Hey, here's your nerdoscope for the week:")
print(f"🐞 You will encounter {nerdoscope.bugs} bugs this week.")
print(f"☕️ You will drink {nerdoscope.coffee_cups} coffee cups this week.")
print("🔮", nerdoscope.reading)

Hey, here's your nerdoscope for the week:
🐞 You will encounter 10 bugs this week.
☕️ You will drink 20 coffee cups this week.
🔮 Attention Software Developer! This week, the stars align to bring you both coding challenges and caffeine cravings. Brace yourself for a whirlwind of bugs and merge conflicts that will test your patience and problem-solving skills. But fear not, for you have the power of your trusty IDE to help you debug your way out of any sticky situation. Remember to take breaks and fuel up with plenty of coffee, for it is the elixir that fuels your coding prowess. Embrace the chaos, for within the chaos lies the opportunity to level up your skills and become a true coding wizard. Good luck, and may your code be bug-free!
